# Primer punto, Parcial 2
## Valentina Espitia
## Miguel García

### a) Convolución

In [1]:
import pandas as pd
import numpy as np

In [17]:
#path = "C:/Users/MCS/OneDrive - Universidad Santo Tomás/Inteligencia Artificial/Codigos Propios/PARCIAL 2 CORTE/"
path = "D:/Downloads/INTELIGENCIA ARTIFICIAL/PARCIAL 2/"
imagen = pd.read_excel(path+"pixeles.xlsx", header=None)
imagen = imagen.values
imagen

array([[0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [18]:
def convolucion(mat_orig, kernel):
    if kernel == "Vertical":
        Kernel = np.array([[1,0,1],[1,0,1],[1,0,1]])
    elif kernel == "Horizontal":
        Kernel = np.array([[1,1,1],[0,0,0],[1,1,1]])
    # Filas resultado --- ((Filas_orig - Filas_kernel)/stride(Saltos)) + 1, si es decimal se redondea hacia abajo.
    # Columnas resultado --- ((Col_orig - Col_kernel)/stride(Saltos)) + 1, si es decimal se redondea hacia abajo.
    filas = ((mat_orig.shape[0] - Kernel.shape[0])/1) + 1
    columnas = ((mat_orig.shape[1] - Kernel.shape[1])/1) + 1
    resultado = np.zeros([int(filas),int(columnas)])
    for k in range(resultado.shape[0]):
        for i in range(resultado.shape[1]):
            resultado[k,i] = (Kernel * mat_orig[(k+0):(3+k),(0+i):(3+i)]).sum()
    return resultado

In [19]:
convolucion(imagen, kernel = "Vertical")

array([[3., 1., 3., 0.],
       [3., 1., 3., 0.],
       [3., 2., 3., 1.],
       [2., 2., 2., 1.]])

In [20]:
convolucion(imagen, kernel = "Horizontal")

array([[2., 2., 2., 0.],
       [3., 3., 2., 0.],
       [3., 4., 3., 1.],
       [1., 1., 1., 0.]])

### b) Padding

In [21]:
def padding(mat_orig, n_pad):
    paddin_col = np.zeros([mat_orig.shape[0],n_pad])
    imagen_pad = np.concatenate((mat_orig, paddin_col), axis = 1)
    
    paddin_fil = np.zeros([n_pad,imagen_pad.shape[1]])
    imagen_pad = np.concatenate((imagen_pad, paddin_fil), axis = 0)
    
    paddin_col = np.zeros([imagen_pad.shape[0],n_pad])
    imagen_pad = np.concatenate((paddin_col,imagen_pad), axis = 1)
    
    paddin_fil = np.zeros([n_pad,imagen_pad.shape[1]])
    imagen_pad = np.concatenate((paddin_fil,imagen_pad), axis = 0)
    return imagen_pad

In [26]:
imagen_pad = padding(imagen,1)
imagen_pad

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

### c) Stride

In [27]:
def stride(mat_orig, kernel, saltos):
    if kernel == "Vertical":
        Kernel = np.array([[1,0,1],[1,0,1],[1,0,1]])
    elif kernel == "Horizontal":
        Kernel = np.array([[1,1,1],[0,0,0],[1,1,1]])
    
    filas = ((mat_orig.shape[0] - Kernel.shape[0])/saltos) + 1
    columnas = ((mat_orig.shape[1] - Kernel.shape[1])/saltos) + 1
    resultado = np.zeros([int(filas),int(columnas)])
        
    for k in range(resultado.shape[0]):
        for i in range(resultado.shape[1]):
            resultado[k,i] = (Kernel * mat_orig[(saltos*k+0):(3+saltos*k),(0+saltos*i):(3+saltos*i)]).sum()
    return resultado

In [31]:
result_1 = stride(imagen, "Vertical", 2)
result_2 = stride(imagen, "Horizontal", 2)

In [29]:
print(f"Para detectar bordes verticales con 2 saltos es: {result_1}")
print(f"Para detectar bordes horizontales con 2 saltos es: {result_2}")

Para detectar bordes verticales con 2 saltos es: [[3. 3.]
 [3. 3.]]
Para detectar bordes horizontales con 2 saltos es: [[2. 2.]
 [3. 3.]]


### d) Stacking

In [99]:
def stacking(mat_orig, n_mapas):
    mapas = []
    for _ in range(n_mapas):
        tipo = np.random.choice(['horizontal', 'vertical'])
        kernel = np.zeros((3, 3))
        if tipo == 'horizontal':
            kernel[0, :] = np.random.randint(1, 5, 3)
            kernel[2, :] = np.random.randint(1, 5, 3)
        else:
            kernel[:, 0] = np.random.randint(1, 5, 3) 
            kernel[:, 2] = np.random.randint(1, 5, 3)
        
        filas = (mat_orig.shape[0] - kernel.shape[0]) + 1
        columnas = (mat_orig.shape[1] - kernel.shape[1]) + 1
        resultado = np.zeros((filas, columnas))
    
        for k in range(filas):
            for i in range(columnas):
                matriz = mat_orig[k:k + kernel.shape[0], i:i + kernel.shape[1]]
                resultado[k, i] = (kernel * matriz).sum()
        
        mapas.append(resultado)
    
    return np.array(mapas)

In [100]:
mapas = stacking(imagen,6)
for i, mapa in enumerate(mapas):
    print(f"Mapa de características {i + 1}:\n{mapa}\n")

Mapa de características 1:
[[ 5.  4. 11.  0.]
 [ 5.  3. 11.  0.]
 [ 5.  5. 11.  4.]
 [ 4.  6.  7.  4.]]

Mapa de características 2:
[[9. 2. 7. 0.]
 [9. 2. 7. 0.]
 [9. 4. 7. 3.]
 [8. 6. 4. 2.]]

Mapa de características 3:
[[6. 1. 3. 0.]
 [6. 1. 3. 0.]
 [6. 2. 3. 1.]
 [5. 4. 2. 1.]]

Mapa de características 4:
[[6. 3. 5. 0.]
 [7. 4. 5. 0.]
 [8. 9. 7. 4.]
 [4. 1. 1. 0.]]

Mapa de características 5:
[[8. 3. 7. 0.]
 [8. 3. 7. 0.]
 [8. 3. 7. 1.]
 [6. 7. 6. 3.]]

Mapa de características 6:
[[ 6.  6.  7.  0.]
 [ 8.  9.  7.  0.]
 [10. 13. 11.  4.]
 [ 3.  2.  3.  0.]]



### e) Max Pooling

In [116]:
def max_pooling(mat_orig, saltos):
    kernel = 2
    filas = (mat_orig.shape[0] - kernel) // saltos + 1
    columnas = (mat_orig.shape[1] - kernel) // saltos + 1
    resultado = np.zeros((filas, columnas))
    
    for k in range(filas):
        for i in range(columnas):
            matriz = mat_orig[k * saltos:k * saltos + kernel, i * saltos:i * saltos + kernel]
            resultado[k, i] = np.max(matriz)
    
    return resultado


In [117]:
max_pooling(imagen,3)

array([[1., 0.],
       [1., 1.]])